### Create Street Intersection Point Feature Class in WGS84
- Requires a line featureclass with street names
- ```FeatureVerticesToPoints_management()``` [Available with Advanced license](http://pro.arcgis.com/en/pro-app/tool-reference/data-management/feature-vertices-to-points.htm)

#### Using [FeatureVerticesToPoints_management()](http://pro.arcgis.com/en/pro-app/tool-reference/data-management/feature-vertices-to-points.htm) will create a point at ```BOTH_ENDS``` for each line representing the street
#### [FindIdentical_management()](http://pro.arcgis.com/en/pro-app/tool-reference/data-management/find-identical.htm) will create the table from the point featureclass created above. The identical ```fields``` is simply the SHAPE of the point created where one street segment ends and another street segment begins.

Looping through a list created will exlclude points that are not representing the intersection of a street with a different name

New Capability: Adding fields to support hyperlinks to Google Streetview and Bing BirdsEye

In [ ]:
import arcpy
import numpy as np
import pandas as pd

In [ ]:
sourceStreets = r'C:\GISData\Data\Snapshot\mxBaseMap.geodatabase\main.Cartographic\main.Streets'
destGDB = r'C:\Users\friendde\Documents\ArcGIS\Projects\NAStreets\NAStreets.gdb'
stIntersectionWGS84 = r'C:\Users\friendde\Documents\ArcGIS\Projects\NAStreets\NAStreets.gdb\StreetIntersectionWGS84'
stVerticesWGS84 = r'C:\Users\friendde\Documents\ArcGIS\Projects\NAStreets\NAStreets.gdb\StreetVerticesWGS84'
identEndsWGS84 = r'C:\Users\friendde\Documents\ArcGIS\Projects\NAStreets\NAStreets.gdb\IdenticalStreetEndsWGS84'
fldNames = {'IntersectingStreets':'Intersecting Streets','StreetName1':'Street Name 1','StreetName2':'Street Name 2','StreetName3':'Street Name 3','StreetName4':'Street Name 4'}
outFlds =['IntersectingStreets','StreetName1','StreetName2','StreetName3','StreetName4','POINT_X','POINT_Y']

In [ ]:
arcpy.env.workspace = destGDB
for fc in arcpy.ListFeatureClasses():
    arcpy.Delete_management(fc)
for tbl in arcpy.ListTables():
    arcpy.Delete_management(tbl)

In [ ]:
sr = arcpy.SpatialReference(4326)
sr.name

In [ ]:
#sr = arcpy.Describe(sourceStreets).spatialReference
arcpy.CreateFeatureclass_management(destGDB,'StreetIntersection','POINT',spatial_reference=sr,out_alias='Steet Intersection')
for fName,fAlias in fldNames.items():
    #print(fName,fAlias)
    arcpy.AddField_management (stIntersection,fName,'TEXT',field_length=100,field_alias=fAlias,)

In [ ]:
arcpy.FeatureVerticesToPoints_management(sourceStreets,stVertices,'BOTH_ENDS')
arcpy.FindIdentical_management(stVertices,identEnds,'SHAPE',output_record_option='ONLY_DUPLICATES')
lastFeatSeq = [row for row in arcpy.da.SearchCursor(identEnds, "FEAT_SEQ")][-1]
lastFeatSeq = lastFeatSeq[0]+1
print(f"number of intersections: {lastFeatSeq}")

In [ ]:
for i in range(1,lastFeatSeq,1):
    FID = []
    streetIntersection = []
    with arcpy.da.SearchCursor(identEnds,["IN_FID","FEAT_SEQ"],f"FEAT_SEQ = {i}") as sc:
        for fid in sc:
            FID.append(fid[0])
        for oid in FID:
            with arcpy.da.SearchCursor(stVertices,["OID@","Street_Label","SHAPE@"],f'OBJECTID = {oid}') as stCur:
                for st in stCur:
                    if len(st[1])>2:
                        streetIntersection.append(st[1])
        # Convert list to set and then back to list to remove duplicate street label names
        streetIntersect = list(set(streetIntersection))
        streetIntersect.sort()
        #print(f"sorted list {streetIntersect} length {len(streetIntersect)}")
        # convert list to string
        stringIntersect = '<->'.join(streetIntersect)
        if len(streetIntersect) == 2:
            #print(f"{stringIntersect}")
            ic = arcpy.da.InsertCursor(stIntersection,["IntersectingStreets","StreetName1","StreetName2","SHAPE@"])
            row = [stringIntersect,streetIntersect[0],streetIntersect[1],st[2]]
            ic.insertRow(row)
            del ic
        elif len(streetIntersect) == 3:
            #print(f"{stringIntersect}")
            ic = arcpy.da.InsertCursor(stIntersection,["IntersectingStreets","StreetName1","StreetName2","StreetName3","SHAPE@"])
            row = [stringIntersect,streetIntersect[0],streetIntersect[1],streetIntersect[2],st[2]]
            ic.insertRow(row)
            del ic
        elif len(streetIntersect) == 4:
            #print(f"{stringIntersect}")
            ic = arcpy.da.InsertCursor(stIntersection,["IntersectingStreets","StreetName1","StreetName2","StreetName3","StreetName4","SHAPE@"])
            row = [stringIntersect,streetIntersect[0],streetIntersect[1],streetIntersect[2],streetIntersect[3],st[2]]
            ic.insertRow(row)
            del ic
        else:
            pass
            #print(f"string list other {stringIntersect}")
            #ic = arcpy.da.InsertCursor(stIntersection,["IntersectingStreets","SHAPE@"])
            #row = [stringIntersect,st[2]]
            #ic.insertRow(row)
            #del ic

In [ ]:
lastIntersect = [row for row in arcpy.da.SearchCursor(stIntersection, "OID@")][-1]
print(f"number of intersections: {lastIntersect}")

In [ ]:
arcpy.AddXY_management(stIntersection)

Using [arcpy.da.TableToNumPyArray()](http://pro.arcgis.com/en/pro-app/arcpy/data-access/tabletonumpyarray.htm) and [pandas.DataFrame()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) display the first 25 rows of the Street Intersection point featureclass

In [ ]:
arr = arcpy.da.TableToNumPyArray(stIntersection,outFlds)
df = pd.DataFrame(arr)
#df['StreetView'] = pd.Series(index=df.index) 
#df.count()
df.head(25)

In [ ]:
df.head()

http://maps.google.com/?cbll=%VALUE%&cbp=12,90,0,0,5&layer=c

df.drop(['StreetView'], axis=1, inplace=True)

In [ ]:
df = df.astype({'POINT_X':str}, copy=False)
df = df.astype({'POINT_Y':str}, copy=False)

In [ ]:
df.dtypes

In [ ]:
#df = df.assign(StreetView='http://maps.google.com/?cbll=' + df["POINY_Y"] + df["POINT_X"] + '&cbp=12,90,0,0,5&layer=c')
df = df.assign(StreetView='http://maps.google.com/?cbll=' + df.POINT_Y + ',' + df.POINT_X + '&cbp=12,90,0,0,5&layer=c')

In [ ]:
df.StreetView.head()

In [ ]:
arcpy.env.workspace = r'C:\Users\friendde\Documents\ArcGIS\Projects\NAStreets'
arcpy.env.overwriteOutput = True

In [ ]:
arcpy.ExportXMLWorkspaceDocument_management (stIntersection, r'C:\Users\friendde\Documents\ArcGIS\Projects\NAStreets\StreetIntersectionWGS84.xml', 'DATA')